In [1]:
import pandas as pd
import numpy as np

## Exercises 1

#### Run `python -m pip install pymysql` from your terminal to install the mysql client (any folder is fine)

done.

#### cd into your exercises folder for this module and run `echo env.py >> .gitignore`

done.

#### Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [2]:
# from env.py take the variables user, password, host
from env import user, password, host

In [4]:
import os
env_exists = os.path.exists('env.py')

In [5]:
env_exists

True

In [6]:
# define a function called get_db_url
# will take in the user, host, password, and schema
# all of these will be string literals
# def get_db_url(user, host, password, db):
    # url = f'mysql+pymysql://{user}:{password}@{host}/{db}'
    # return url

# if I made this in my env file:
# I may make this a little easier on myself by formatting it this 
# way:
def create_url(db,
               user=user, 
               host=host, 
               password=password,
              env_exists=os.path.exists('env.py')):
    '''
    create_url will take in string values for
    user: a username (string literal)
    host: a host server dns or ip (string literal)
    password: a credential sequence (string literal)
    db: a mysql schema
    
    return: a formatted string that contains your formatted
    connection.  Don't put it in plaintext :)
    '''
    if env_exists:
        return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    else:
        return 'hey I think you need credentials!'

In [7]:
my_connection = create_url('employees')

In [8]:
# if it gave me my else return we would see the word hey in there
'hey' in my_connection

False

#### Use your function to obtain a connection to the employees database.

In [ ]:
# see above

#### Once you have successfully run a query:

In [9]:
query = 'SELECT * FROM employees LIMIT 10'

In [10]:
# assign df_test the results of the query and connection
df_test = pd.read_sql(query, my_connection)

In [11]:
df_test

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
5,10006,1953-04-20,Anneke,Preusig,F,1989-06-02
6,10007,1957-05-23,Tzvetan,Zielinski,F,1989-02-10
7,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15
8,10009,1952-04-19,Sumant,Peac,F,1985-02-18
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


#### a. Intentionally make a typo in the database url. What kind of error message do you see?

In [12]:
my_connection = create_url('bemployees')
pd.read_sql(query, my_connection)

OperationalError: (pymysql.err.OperationalError) (1044, "Access denied for user 'ada_674'@'%' to database 'bemployees'")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

#### b. Intentionally make an error in your SQL query. What does the error message look like?

In [13]:
# fix the connection back to where it was
my_connection = create_url('employees')

In [15]:
pd.read_sql('select *, from employees limit 5', my_connection)

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'from employees limit 5' at line 1")
[SQL: select *, from employees limit 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

#### Read the employees and titles tables into two separate DataFrames.

In [ ]:
# schema: is going to be employees for both instances

In [ ]:
# url: will be the same because the schema is the same
# my_connection can remain

In [16]:
# queries for titles and employees
# grab everything for each, no limitations
employees = pd.read_sql('SELECT * FROM employees', my_connection)
titles = pd.read_sql('SELECT * FROM titles', my_connection)

#### How many rows and columns do you have in each DataFrame? Is that what you expected?

In [18]:
employees.shape

(300024, 6)

In [19]:
titles.shape

(443308, 4)

In [21]:
employees.head(2)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21


#### Display the summary statistics for each DataFrame.

In [22]:
# use describe!
titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


In [23]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB


In [25]:
employees.drop(columns='emp_no').describe()

,birth_date,first_name,last_name,gender,hire_date
count,300024,300024,300024,300024,300024
unique,4750,1275,1637,2,5434
top,1952-03-08,Shahab,Baba,M,1985-06-20
freq,95,295,226,179973,132


#### How many unique titles are in the titles DataFrame?

In [27]:
titles.drop(columns='emp_no'
           ).describe(
)['title']['unique']

7

In [31]:
# get the distinct titles
# get the length of the number of titles in the series
len(titles.title.unique())

7

In [30]:
# same thing here for the value_counts
len(titles.title.value_counts())

7

In [32]:
# or use nunique for the number of unique vals
titles.title.nunique()

7

#### What is the oldest date in the to_date column?

In [51]:
titles_non_current = pd.read_sql(
    'SELECT * FROM titles WHERE to_date < NOW()', my_connection)

In [52]:
titles_non_current

,emp_no,title,from_date,to_date
0,10004,Engineer,1986-12-01,1995-12-01
1,10005,Staff,1989-09-12,1996-09-12
2,10007,Staff,1989-02-10,1996-02-11
3,10008,Assistant Engineer,1998-03-11,2000-07-31
4,10009,Assistant Engineer,1985-02-18,1990-02-18
...,...,...,...,...
203179,499992,Staff,1987-05-10,1992-05-09
203180,499994,Engineer,1993-02-22,1993-10-27
203181,499996,Engineer,1996-05-13,2002-05-13
203182,499997,Engineer,1987-08-30,1992-08-29


In [46]:
# whats going on with this mess?
# dataframe titles
titles[
    #where
    #not: ~
    # to_date as a string
    #because im making a string comparison
    #apply: which throws a function at it
    #lambda function: in-line definition
    #look at the cell, see if 9999 is in it
    #return a boolean
    #use that as a mask
    #do the opposite with the ~ from above
    ~titles.to_date.astype(str
                          ).apply(
        lambda x: '9999' in x)]

,emp_no,title,from_date,to_date
3,10004,Engineer,1986-12-01,1995-12-01
6,10005,Staff,1989-09-12,1996-09-12
9,10007,Staff,1989-02-10,1996-02-11
10,10008,Assistant Engineer,1998-03-11,2000-07-31
11,10009,Assistant Engineer,1985-02-18,1990-02-18
...,...,...,...,...
443297,499992,Staff,1987-05-10,1992-05-09
443299,499994,Engineer,1993-02-22,1993-10-27
443301,499996,Engineer,1996-05-13,2002-05-13
443303,499997,Engineer,1987-08-30,1992-08-29


In [53]:
# if you want to really make sure something
# is being rendered as a datetime in pandas:
# (we're going to use this later in TSA)
# tpd.to_datetime(titles.to_date.).min()

In [54]:
titles.to_date.min()

datetime.date(1985, 3, 1)

In [36]:
# if we try to do subtraction here:
# we note that these are already in datetime
# format
# which means we dont need to do any conversion 
# measures
titles.to_date - titles.from_date

0         2926512 days, 0:00:00
1         2922821 days, 0:00:00
2         2923065 days, 0:00:00
3            3287 days, 0:00:00
4         2923067 days, 0:00:00
                  ...          
443303       1826 days, 0:00:00
443304    2924256 days, 0:00:00
443305    2921945 days, 0:00:00
443306       1826 days, 0:00:00
443307    2922337 days, 0:00:00
Length: 443308, dtype: object

#### What is the most recent date in the to_date column?

In [55]:
titles['to_date'].max()

datetime.date(9999, 1, 1)

In [60]:
titles_non_current['to_date'].max()

datetime.date(2002, 8, 1)